In [ ]:
import tensorflow_decision_forests as tfdf
import tensorflow_probability as tfp
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from scipy.stats import pearsonr
def ubrmse(ground,pred):
    bias = np.mean(ground-pred)
    rmse = np.sqrt(np.mean((ground-pred)**2))
    ubrmse = np.sqrt(rmse**2-bias**2)
    return round(ubrmse,4)

# Load Models

## Random Forest

This section loads the RF ensemble members into the rf_dict and relevant functions for ensemble predictions

In [ ]:
import os
rf_dict = {}
for i in os.listdir('Models/RF/'):
    if i in rf_dict.keys():
        continue
    if i =='summaries':
        continue
    try:
        int(i)
    except:
        continue
    rf_dict[i] = tf.keras.models.load_model('Models/RF/{}'.format(i))

In [ ]:
def batch_dict(df,spatial):
    test_dict = {}
    lst,ndvi = spatial
    for var in ['sand','clay','ph','dem','ET',ndvi,lst,'smap', 'precip']:
        if var == ndvi:
            test_dict['NDVI_500'] = df[var].values
        elif var == lst:
            test_dict['LST_11'] = df[var].values
        else:
            test_dict[var] = df[var].values
    return test_dict

def rf_predict(df,model_dict,spatial):
    batched_dict = batch_dict(df,spatial)
    preds = []
    for i in model_dict.values():
        preds.append(i.predict_on_batch(batched_dict))
    preds = np.asarray(preds)
    predictions = np.mean(preds,axis=0)
    return predictions.squeeze()

## Ensembles

This is the ensemble prediction functions and nll loss needed when loading a model

In [ ]:
def ens_pred(df,model_dict):
    preds = []
    for i in model_dict.values():
        preds.append(i(df.values).mean().numpy().squeeze())
    
    prediction = np.mean(np.asarray(preds),axis=0).squeeze()
    return prediction

def wdl_ens(df,model_dict,spatial=False):
    wide_in,dnn_in = wide_inputs(df,spatial)
    preds = []
    for i in model_dict.values():
        preds.append(i((wide_in,dnn_in)).numpy().squeeze())
    return np.mean(np.asarray(preds),axis=0)

cat_dict = {'texture':13,'mcd12':18,'koep':32}

def wide_inputs(df,spatial=False):
    if spatial:
        dnn_in = df.loc[:,['sand','clay','ph','dem','NDVI_250','LST_11','ET','precip','smap']]
    else:
        dnn_in = df.loc[:,['sand','clay','ph','dem','NDVI_500','LST_11','ET','precip','smap']]
    wide_in = dnn_in
    cats = df.loc[:,['texture','mcd12','koep']]
    cat_embeds = []
    for cat in cats.columns:
        embedding = np.eye(cat_dict[cat])[cats[cat].astype(int)]
        cat_embeds.append(np.asarray(embedding))

    embeddings = np.concatenate(cat_embeds,axis=1)
    dnn_in = np.concatenate([dnn_in,embeddings],axis=1)
    return wide_in.values.astype(float),dnn_in.astype(float)

def nll(y_true, y_pred):
    return -y_pred.log_prob(y_true)

In [ ]:
prob_dict = {}
dense_dict = {}
wdl_dict = {}

for att in ['sand','clay','koep','mcd12','free','ph','texture']:
    dense_dict[att] = tf.keras.models.load_model(f'Models/Dense/{att}',custom_objects={'nll':nll}, compile=False)
    prob_dict[att] = tf.keras.models.load_model(f'Models/Prob/{att}',custom_objects={'nll':nll}, compile=False)
    wdl_dict[att] = tf.keras.models.load_model(f'Models/WDL/{att}',custom_objects={'nll':nll}, compile=False)

# Validation Datasets

In [ ]:
def to_rf(df):
    df = df.fillna(-1)
    for col in df.columns:
        if col in ['dem','smap','precip','in_situ']:
            df[col] = df[col].astype('float32')
        elif col =='date':
            continue
        else:
            df[col] = df[col].astype('int16')
    return df

def station_metrics(df,pred,situ='in_situ'):
    let = pred.split('_')[0]
    stat_dicts = {}
    for stat in df.index.unique():
        stat_df = df.loc[stat]
        if type(stat_df) == pd.core.series.Series:
            continue
        stat_r = pearsonr(stat_df[situ],stat_df[pred])[0]
        stat_ub = ubrmse(stat_df[situ],stat_df[pred])
        stat_b = round(np.mean(stat_df[pred]-stat_df[situ]),3) 
        stat_dicts[stat] = {'{}_r'.format(let):stat_r,
                            '{}_ub'.format(let):stat_ub,
                            '{}_b'.format(let):stat_b}
    return pd.DataFrame().from_dict(stat_dicts,orient='index')

## Val DF

Here we load the datasets

In [ ]:
test_df = pd.read_pickle('Datasets/val_df.pkl')
test_rf = pd.read_pickle('Datasets/RF_val_df.pkl')

Here we predict SWC

In [ ]:
variables = ['sand','clay','ph','dem','ET','NDVI_500','LST_11','precip','smap']
val_df = test_df.loc[:,variables]
dense_pred = ens_pred(val_df,dense_dict)
prob_pred = ens_pred(val_df,prob_dict)
wens_pred = wdl_ens(test_df,wdl_dict)
rf_pred = rf_predict(test_rf.drop('in_situ',axis=1),rf_dict,['LST_11','NDVI_500'])

Here we save predictions to a DataFrame

In [ ]:
val_pred_df = test_df.copy()[['in_situ','smap']]
val_pred_df['d_pred'] = dense_pred
val_pred_df['p_pred'] = prob_pred
val_pred_df['wens_pred'] = wens_pred
val_pred_df['r_pred'] = rf_pred

#Save Predictions to File
val_pred_df.to_pickle('Datasets/val_df_preds.pkl')

Here we go through each station in the dataset and calculate the metrics for each station and combine them into a single dataframe

In [ ]:
pred_df = val_pred_df
prob_mets = station_metrics(pred_df,'p_pred')
dense_mets = station_metrics(pred_df,'d_pred')
smap_mets = station_metrics(pred_df,'smap')
wens_mets = station_metrics(pred_df,'wens_pred')
rf_mets = station_metrics(pred_df,'r_pred')

ds_mets = pd.concat([dense_mets,prob_mets,wens_mets,rf_mets,smap_mets],axis=1).dropna()
algs = ['d_','p_','wens_','r_','smap_']

#List the Station Metrics for the top 16 stations
stations_df = ds_mets.loc[test_df.loc[ds_mets.index].index.value_counts().index][:16]
stations_df = pd.concat([stations_df,stations_df.mean().rename('Avg.').to_frame().T])
stations_df.round(3).style.highlight_max(subset=[i+'r' for i in algs],axis=1,color='blue').highlight_min(subset=[i+'ub' for i in algs],axis=1,color='red').highlight_min(subset=[i+'b' for i in algs],axis=1,color='green').format(precision=3)

This is a summary of the metrics scored on each station in the dataset

In [ ]:
series = []
for var in ['_r','_ub','_b']:
    sub = ds_mets[[i for i in ds_mets.columns if var in i]]
    avg = sub.mean()
    avg.name = var
    avg = avg.to_frame().T
    avg.columns = ['Dense','Prob','Wens','RF','SMAP']
    series.append(avg)
avgs = pd.concat(series)
avgs.index = ['R','ubRMSE','Bias']
avgs.round(3)

## Washita

In [ ]:
wa_df = pd.read_pickle('Datasets/wa_validation.pkl')
wa_rf = wa_df.copy()
wa_df['NDVI_500'] /= 10000
wa_df['LST_11'] /= 5000
wa_df['LST_11'] -= 2.7315
wa_df['precip'] /= 1000
wa_df['sand'] /= 100
wa_df['clay'] /= 100
wa_df['ET'] /= 1000
wa_df['ph'] /= 1000
wa_df['dem'] /= 10000
wa_df.dem = round(wa_df.dem,4)
cond = wa_df.LST_11 > 0
wa_df = wa_df[cond]
wa_rf = wa_rf[cond]
wa_rf = to_rf(wa_rf)

In [ ]:
val_df = wa_df.loc[:,['sand','clay','ph','dem','ET','NDVI_500','LST_11','precip','smap']]
dense_pred = ens_pred(val_df,dense_dict)
prob_pred = ens_pred(val_df,prob_dict)
wens_pred = wdl_ens(wa_df,wdl_dict)
rf_pred = rf_predict(wa_rf,rf_dict,['LST_11','NDVI_500'])

In [ ]:
wa_pred_df = wa_df.copy()[['in_situ','smap']]
wa_pred_df['d_pred'] = dense_pred
wa_pred_df['p_pred'] = prob_pred
wa_pred_df['wens_pred'] = wens_pred
wa_pred_df['r_pred'] = rf_pred
wa_pred_df.to_pickle('Datasets/wa_df_preds.pkl')

In [ ]:
prob_mets = station_metrics(wa_pred_df,'p_pred')
dense_mets = station_metrics(wa_pred_df,'d_pred')
smap_mets = station_metrics(wa_pred_df,'smap')
wens_mets = station_metrics(wa_pred_df,'wens_pred')
r_mets = station_metrics(wa_pred_df,'r_pred')
wa_mets = pd.concat([dense_mets,prob_mets,wens_mets,r_mets,smap_mets],axis=1).dropna()

In [ ]:
new_mets = wa_mets
mean_mets = new_mets.mean().rename('Avg.')

# Concatenate the selected rows with the mean
new_mets = pd.concat([new_mets.loc[new_mets.index.value_counts().index], mean_mets.to_frame().T])
algs = ['p_','d_','wens_','r_','smap_']

new_mets.round(3).style.highlight_max(subset=[i+'r' for i in algs],axis=1,color='blue').highlight_min(subset=[i+'ub' for i in algs],axis=1,color='red').highlight_min(subset=[i+'b' for i in algs],axis=1,color='green').format(precision=3)

In [ ]:
series = []
for var in ['_r','_ub','_b']:
    sub = wa_mets[[i for i in wa_mets.columns if var in i]]
    avg = sub.mean()
    avg.name = var
    avg = avg.to_frame().T
    avg.columns = ['Dense','Prob','Wens','RF','SMAP']
    series.append(avg)
avgs = pd.concat(series)
avgs.index = ['R','ubRMSE','Bias']
avgs.round(3)

## Fort Cobb

In [ ]:
fc_df = pd.read_pickle('Datasets/fc_validation.pkl')
fc_rf = fc_df.copy()
fc_df['NDVI_500'] /= 10000
fc_df['LST_11'] /= 5000
fc_df['LST_11'] -= 2.7315
fc_df['precip'] /= 1000
fc_df['sand'] /= 100
fc_df['clay'] /= 100
fc_df['ET'] /= 1000
fc_df['ph'] /= 100
fc_df['dem'] /= 10000
fc_df.dem = round(fc_df.dem,4)
cond = fc_df.LST_11 > 0
fc_df = fc_df[cond]
fc_rf = fc_rf[cond]
fc_rf = to_rf(fc_rf)

In [ ]:
base_df = fc_df
val_df = base_df.loc[:,['sand','clay','ph','dem','ET','NDVI_500','LST_11','precip','smap']]
dense_pred = ens_pred(val_df,dense_dict)
prob_pred = ens_pred(val_df,prob_dict)
wens_pred = wdl_ens(base_df,wdl_dict)
rf_pred = rf_predict(fc_rf,rf_dict,['LST_11','NDVI_500'])

In [ ]:
fc_pred_df = fc_df.copy()[['in_situ','smap']]
fc_pred_df['d_pred'] = dense_pred
fc_pred_df['p_pred'] = prob_pred
fc_pred_df['wens_pred'] = wens_pred
fc_pred_df['r_pred'] = rf_pred
fc_pred_df.to_pickle('Datasets/fc_df_preds.pkl')

In [ ]:
pred_df = fc_pred_df
prob_mets = station_metrics(pred_df,'p_pred')
dense_mets = station_metrics(pred_df,'d_pred')
smap_mets = station_metrics(pred_df,'smap')
wens_mets = station_metrics(pred_df,'wens_pred')
r_mets = station_metrics(pred_df,'r_pred')
fc_mets = pd.concat([dense_mets,prob_mets,wens_mets,r_mets,smap_mets],axis=1).dropna()

In [ ]:
new_mets = fc_mets
mean_mets = new_mets.mean().rename('Avg.')

# Concatenate the selected rows with the mean
new_mets = pd.concat([new_mets.loc[new_mets.index.value_counts().index], mean_mets.to_frame().T])
algs = ['p_','d_','wens_','r_','smap_']

new_mets.round(3).style.highlight_max(subset=[i+'r' for i in algs],axis=1,color='blue').highlight_min(subset=[i+'ub' for i in algs],axis=1,color='red').highlight_min(subset=[i+'b' for i in algs],axis=1,color='green').format(precision=3)

In [ ]:
series = []
for var in ['_r','_ub','_b']:
    sub = fc_mets[[i for i in fc_mets.columns if var in i]]
    avg = sub.mean()
    avg.name = var
    avg = avg.to_frame().T
    avg.columns = ['Dense','Prob','Wens','RF','SMAP']
    series.append(avg)
avgs = pd.concat(series)
avgs.index = ['R','ubRMSE','Bias']
avgs.round(3)